## Mapping a united-atom (UA) atomistic topology to a coarse-grained topology

In this example, we're going to learn how to use the TopologyMapper class to map a united-atom atomistic topology to a coarse-grained one.

In [1]:
import mdtraj as md

from cgtools.mapping import TopologyMapper

> Preparing to load module geomlib
> Compiling binary for source of module geomlib
> Finished loading module geomlib (took 2.709 s)
> Preparing to load module maplib
> Compiling binary for source of module maplib
> Finished loading module maplib (took 0.887 s)
> openmm or mdtraj did not load; these features will not work!


Initialize a `TopologyMapper` object using either an `mdtraj.Topology` object or a path to a pdb file. The topology in this pdb file has two dodecane molecules and two poly butyl acrylate (PBA) 5-mers, 

In [2]:
# using pdb file
tm = TopologyMapper("solution.pdb")

# using mdtraj.Topology object
top = md.load("solution.pdb").topology
tm = TopologyMapper(top)

Mapping a UA topology is similar to mapping an explicit-hydrogen topology, but because UA topologies use non-conventional atom types, we have to manually set masses of those atoms using the `set_atom_mass` method.

In [3]:
tm.set_atom_mass('_CH', 13.019)
tm.set_atom_mass('_CH2', 14.027)
tm.set_atom_mass('_CH3', 15.039)

For the dodecane molecules, which are named `DOD` in the pdb file, we map them to a 3-bead model.

In [4]:
tm.add_residue_map('DOD',
                   ['C4', 'C4', 'C4'],
                   n_heavy_atoms_per_cg_bead=[4, 4, 4])

For the poly butyl acrylate 5-mers, each chain has 5 residues named `A4`, each of which represent a monomer in the chain.
We map each monomer to two beads, where the acrylate group is represented by a bead named `Bpba` and the acrylate bead is represented by the same `C4` bead used in the dodecane.

In [5]:
tm.add_residue_map('A4',
                   ['Bpba', 'C4'],
                   n_heavy_atoms_per_cg_bead=[5, 4])

Once all of the mappings have been added, we can call the `create_map` method to create the mapping for the topology.

In [6]:
tm.create_map()

After the mapping is created, we can access information about the CG system with the attributes `cg_beads` and `cg_bonds`.
The `cg_beads` attributes returns an iterator that iterates through `CGBead` objects. For each `CGBead` object, the `index` attribute indicates the index of the bead in the CG topology, the `name` attribute gives the name of the bead, and the `aa_indices` and `aa_masses` attributes give the indices of atoms in the AA topology that map to the bead and the masses of the atomistic atoms.

In [7]:
for bead in tm.cg_beads:
    print(f"Bead {bead.index} : {bead.name}")
    print(f"    AA indices : {bead.aa_indices}")
    print(f"    AA masses  : {bead.aa_masses}")
    print(" ")

Bead 0 : Bpba
    AA indices : [0 1 2 3 4]
    AA masses  : [15.039   13.019   12.01078 15.99943 15.99943]
 
Bead 1 : C4
    AA indices : [5 6 7 8]
    AA masses  : [14.027 14.027 14.027 15.039]
 
Bead 2 : Bpba
    AA indices : [ 9 10 11 12 13]
    AA masses  : [14.027   13.019   12.01078 15.99943 15.99943]
 
Bead 3 : C4
    AA indices : [14 15 16 17]
    AA masses  : [14.027 14.027 14.027 15.039]
 
Bead 4 : Bpba
    AA indices : [18 19 20 21 22]
    AA masses  : [14.027   13.019   12.01078 15.99943 15.99943]
 
Bead 5 : C4
    AA indices : [23 24 25 26]
    AA masses  : [14.027 14.027 14.027 15.039]
 
Bead 6 : Bpba
    AA indices : [27 28 29 30 31]
    AA masses  : [14.027   13.019   12.01078 15.99943 15.99943]
 
Bead 7 : C4
    AA indices : [32 33 34 35]
    AA masses  : [14.027 14.027 14.027 15.039]
 
Bead 8 : Bpba
    AA indices : [36 37 38 39 40]
    AA masses  : [14.027   14.027   12.01078 15.99943 15.99943]
 
Bead 9 : C4
    AA indices : [41 42 43 44]
    AA masses  : [14.027 14.

When iterating through CG bonds, each item is a tuple of `CGBead` objects.

In [8]:
for bond in tm.cg_bonds:
    print(bond[0].index, bond[1].index)

0 2
0 1
2 4
2 3
4 6
4 5
6 8
6 7
8 9
10 12
10 11
12 14
12 13
14 16
14 15
16 18
16 17
18 19
20 21
21 22
23 24
24 25


If the `sim` module, which is developed by the Shell group at UCSB, is available, then the `TopologyMapper` object can be converted to objects used by `sim`.

In [9]:
# try importing sim
try:
    import sim
    print("sim module found")
    print(" ")
except ModuleNotFoundError:
    sim = None
    print("sim module not found")
    
if sim is not None:
    # convert to sim
    sim_topology = tm.to_sim()
    AtomNames = sim_topology.AtomNames
    AtomTypes = sim_topology.AtomTypes
    MolTypes = sim_topology.MolTypes
    PosMap = sim_topology.PosMap
    World = sim_topology.World
    System = sim_topology.System
    
    print(AtomNames)
    print(" ")
    print(AtomTypes)
    print(" ")
    print(MolTypes)

sim module found
 
['Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4']
 
{'Bpba': Bpba, 'C4': C4}
 
[0:[Bpba0, C41, Bpba2, C43, Bpba4, C45, Bpba6, C47, Bpba8, C49], 1:[C40, C41, C42]]
